In [109]:
!python -m spacy download en_core_web_sm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 8.0 MB/s eta 0:00:00m eta 0:00:010:01:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [114]:
import os
os.chdir("/home/congdc/Project/generative-banner")

In [115]:
from PIL import Image
import numpy as np
import cv2
from utils.translate import Translater
import pandas as pd
from core.stable_diffusion import SD_Model
import re
from collections import Counter
import nltk
import underthesea
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import shutil
import transformers
import spacy
DEVICE = "cuda"

In [144]:
TRAIN_PATH  = "./data/train/target/"
INPUT_IMAGES = "./data/train/target/images_hd/"
OUTPUT_IMAGES = "./data/train/target/images_train/"

In [145]:
df_train = pd.read_csv(f"{TRAIN_PATH}info.csv")
id_imgs_train = list(df_train["id"])
captions_train = list(df_train['caption'])
description_train = list(df_train['description'])
moreInfo_train = list(df_train['moreInfo'])
bannerImages_train = list(df_train['bannerImage'])

In [146]:
def clear_data(list_sentence):
    list_result = []
    for sentence in tqdm(list_sentence):
        sentence = str(sentence).lower()
        sentence = re.sub(r'[.\-\/]', ' ', sentence)
        list_result.append(sentence)
    return list_result

In [147]:
def v2e(captions):
    list_result = []
    for idx, caption in tqdm(enumerate(captions)):
        new_caption = Translater.vie2eng(caption)
        
        list_result.append(new_caption)

    return list_result

In [301]:
def replace_nth_occurrence(input_string, target_word, replacement, n):
    # Sử dụng biểu thức chính quy để tìm và thay thế từ chỉ từ lần lặp thứ n trở đi
    pattern = re.compile(r'((?:\b' + re.escape(target_word) + r'\b.*?){' + str(n-1) + r'})\b' + re.escape(target_word) + r'\b')
    result = pattern.sub(r'\1' + replacement, input_string)
    return result

In [307]:
def gen_promt(source1, source2="", source3=""):
    nlp = spacy.load("en_core_web_sm")
    list_promt = []
    for sentence in source1:

        
        sentence = sentence.lower()
        sentence = sentence.replace("but", " ")
        sentence = sentence.replace("also", " ")
        sentence = sentence.replace("-", " ")
        sentence = sentence.replace(".", ",")
        sentence = sentence.replace("en:", "")
        sentence = sentence.replace("+", "")
        
        sentence = sentence.replace(",,,", "")
        sentence = sentence.replace("%", "")
        sentence = sentence.replace("$", "")
        sentence = sentence.replace("!", ",")
        sentence = re.sub(r'\b\d\S*\b', 'x', sentence)
        sentence = re.sub(r'\b(\w+)\s+\1\b', r'\1', sentence)
        sentence = re.sub(r'\s+', ' ', sentence)
        
        
        if sentence[-1] == ",":
            sentence = sentence[:-1]
        list_word = sentence.split(" ")
        counts = Counter(list_word)
        for word, count in counts.items():
            if count > 4:
                # sentence = sentence.replace(word, "")
                sentence = sentence.replace(word, "")
                sentence += word

        sentence = sentence.replace("  ", "")
        list_promt.append(sentence)
    return list_promt

In [305]:
def save_images(name_imgs, list_promt):
    for name_img, promt in tqdm(zip(name_imgs, list_promt)):
        promt = promt.lower()
        
        name_img = name_img.replace('.', '_out.')
        input_image = f"{INPUT_IMAGES}{name_img}"
        output_image = input_image.replace(name_img, f"{promt}.jpg").replace(INPUT_IMAGES, OUTPUT_IMAGES)
        try:
            shutil.copy(input_image, output_image)
        except:
            pass

# Xử lý dữ liệu

1. Làm sạch dữ liệu tiếng việt

In [231]:
captions_train = clear_data(captions_train)
description_train = clear_data(description_train)
moreInfo_train = clear_data(moreInfo_train)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1362/1362 [00:00<00:00, 885269.18it/s]


2. Chuyển đổi ngôn ngữ

In [152]:
captions_train = v2e(captions_train)
# description_train = v2e(description_train)
# moreInfo_train = v2e(moreInfo_train)

1362it [03:09,  7.19it/s]


3. Gen promt

In [308]:
list_promt = gen_promt(captions_train)

In [298]:
# nlp = spacy.load("en_core_web_sm")
# tokens = nlp(captions_train[2])

In [309]:
# for token in tokens:
#     print(token.text, token.pos_, token.dep_)

for i, promt in enumerate(list_promt):
    print(f"line {i}: {promt}")

line 0: x ply aluminum coated car hoods, heat resistant, waterproof, protecting the car from the risk of stealing mirrors 
line 1: the latest floor mats for four seven cars, luxurious, durable, noise reducing 
line 2: enhance the chic elegance of your car with high end car carpeting x 
line 3: all purpose high end car floor carpets x for x x seats, clean, water resistant, noise resistant 
line 4: x layer canvas, heat, dirt, water, scratch protection, full coverage
line 5: three ply car canvas, sun protection, rain protection, burglar protection, mirrors, two minutes of protection 
line 6: usb flash drive for bolero music, lyrical music, youth music, remix to music in x of cars usb drive for cars 
line 7: x layer heat resistant car canvas, scratch resistant, sun resistant, dirt resistant, waterproof 
line 8: heat protection fire protection dust protection with reflective aluminum x ply car coating
line 9: a really essential x floor to floor leather rug, a four seater, seven seater, mult

# Lưu kết quả

In [311]:
save_images(bannerImages_train, list_promt)

1362it [00:00, 6503.00it/s]


In [312]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("NlpHUST/ner-vietnamese-electra-base")
model = AutoModelForTokenClassification.from_pretrained("NlpHUST/ner-vietnamese-electra-base")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "Liên quan vụ việc CSGT bị tố đánh dân, trúng một cháu nhỏ đang ngủ, đang lan truyền trên mạng xã hội, Đại tá Nguyễn Văn Tảo, Phó Giám đốc Công an tỉnh Tiền Giang vừa có cuộc họp cùng Chỉ huy Công an huyện Châu Thành và một số đơn vị nghiệp vụ cấp tỉnh để chỉ đạo làm rõ thông tin."

ner_results = nlp(example)
print(ner_results)

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/411k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/532M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'B-PERSON', 'score': 0.99887997, 'index': 28, 'word': 'Nguyễn', 'start': 109, 'end': 115}, {'entity': 'I-PERSON', 'score': 0.99937624, 'index': 29, 'word': 'Văn', 'start': 116, 'end': 119}, {'entity': 'I-PERSON', 'score': 0.9993292, 'index': 30, 'word': 'Tảo', 'start': 120, 'end': 123}, {'entity': 'B-ORGANIZATION', 'score': 0.99909914, 'index': 35, 'word': 'Công', 'start': 138, 'end': 142}, {'entity': 'I-ORGANIZATION', 'score': 0.99949706, 'index': 36, 'word': 'an', 'start': 143, 'end': 145}, {'entity': 'I-ORGANIZATION', 'score': 0.99938357, 'index': 37, 'word': 'tỉnh', 'start': 146, 'end': 150}, {'entity': 'I-ORGANIZATION', 'score': 0.9992514, 'index': 38, 'word': 'Tiền', 'start': 151, 'end': 155}, {'entity': 'I-ORGANIZATION', 'score': 0.9992137, 'index': 39, 'word': 'Giang', 'start': 156, 'end': 161}, {'entity': 'B-ORGANIZATION', 'score': 0.99908066, 'index': 47, 'word': 'Công', 'start': 191, 'end': 195}, {'entity': 'I-ORGANIZATION', 'score': 0.9994672, 'index': 48, 'word